# Masakhane - Machine Translation for African Languages (Using JoeyNMT)

## Note before beginning:
### - The idea is that you should be able to make minimal changes to this in order to get SOME result for your own translation corpus. 

### - The tl;dr: Go to the **"TODO"** comments which will tell you what to update to get up and running

### - If you actually want to have a clue what you're doing, read the text and peek at the links

### - With 100 epochs, it should take around 7 hours to run in Google Colab

### - Once you've gotten a result for your language, please attach and email your notebook that generated it to masakhanetranslation@gmail.com

### - If you care enough and get a chance, doing a brief background on your language would be amazing. See examples in  [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

## Retrieve your data & make a parallel corpus

If you are wanting to use the JW300 data referenced on the Masakhane website or in our GitHub repo, you can use `opus-tools` to convert the data into a convenient format. `opus_read` from that package provides a convenient tool for reading the native aligned XML files and to convert them to TMX format. The tool can also be used to fetch relevant files from OPUS on the fly and to filter the data as necessary. [Read the documentation](https://pypi.org/project/opustools-pkg/) for more details.

Once you have your corpus files in TMX format (an xml structure which will include the sentences in your target language and your source language in a single file), we recommend reading them into a pandas dataframe. Thankfully, Jade wrote a silly `tmx2dataframe` package which converts your tmx file to a pandas dataframe. 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# TODO: Set your source and target languages. Keep in mind, these traditionally use language codes as found here:
# These will also become the suffix's of all vocab and corpus files used throughout
import os
source_language = "en"
target_language = "yo" 
lc = False  # If True, lowercase the data.
seed = 42  # Random seed for shuffling.
tag = "baseline" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag

# This will save it to a folder in our gdrive instead!
!mkdir -p "/content/drive/My Drive/masakhane/$src-$tgt-$tag"
os.environ["gdrive_path"] = "/content/drive/My Drive/masakhane/%s-%s-%s" % (source_language, target_language, tag)

In [3]:
!echo $gdrive_path

/content/drive/My Drive/masakhane/en-yo-baseline


In [4]:
# Install opus-tools
! pip install opustools-pkg

     |████████████████████████████████| 81kB 5.5MB/s 


In [5]:
# Downloading our corpus
! opus_read -d JW300 -s $src -t $tgt -wm moses -w jw300.$src jw300.$tgt -q

# extract the corpus file
! gunzip JW300_latest_xml_$src-$tgt.xml.gz


Alignment file /proj/nlpl/data/OPUS/JW300/latest/xml/en-yo.xml.gz not found. The following files are available for downloading:

   4 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en-yo.xml.gz
 263 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en.zip
  58 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/yo.zip

 325 MB Total size
./JW300_latest_xml_en-yo.xml.gz ... 100% of 4 MB
./JW300_latest_xml_en.zip ... 100% of 263 MB
./JW300_latest_xml_yo.zip ... 100% of 58 MB


In [6]:
# Download the global test set.
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
  
# And the specific test set for this language pair.
os.environ["trg"] = target_language 
os.environ["src"] = source_language 

! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.en 
! mv test.en-$trg.en test.en
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.$trg 
! mv test.en-$trg.$trg test.$trg

--2020-01-17 17:19:16--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277791 (271K) [text/plain]
Saving to: ‘test.en-any.en’

test.en-any.en      100%[===================>] 271.28K  --.-KB/s    in 0.02s   

2020-01-17 17:19:16 (13.9 MB/s) - ‘test.en-any.en’ saved [277791/277791]

--2020-01-17 17:19:18--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-yo.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Leng

In [7]:
# Read the test data to filter from train and dev splits.
# Store english portion in set for quick filtering checks.
en_test_sents = set()
filter_test_sents = "test.en-any.en"
j = 0
with open(filter_test_sents) as f:
  for line in f:
    en_test_sents.add(line.strip())
    j += 1
print('Loaded {} global test sentences to filter from the training/dev data.'.format(j))

Loaded 3571 global test sentences to filter from the training/dev data.


In [12]:
import pandas as pd

# TMX file to dataframe
source_file = 'jw300.' + source_language
target_file = 'jw300.' + target_language

source = []
target = []
skip_lines = []  # Collect the line numbers of the source portion to skip the same lines for the target portion.
with open(source_file) as f:
    for i, line in enumerate(f):
        # Skip sentences that are contained in the test set.
        if line.strip() not in en_test_sents:
            source.append(line.strip())
        else:
            skip_lines.append(i)             
with open(target_file) as f:
    for j, line in enumerate(f):
        # Only add to corpus if corresponding source was not skipped.
        if j not in skip_lines:
            target.append(line.strip())
    
print('Loaded data and skipped {}/{} lines since contained in test set.'.format(len(skip_lines), i))
    
df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])
# if you get TypeError: data argument can't be an iterator is because of your zip version run this below
#df = pd.DataFrame(list(zip(source, target)), columns=['source_sentence', 'target_sentence'])
df.head(5)

Loaded data and skipped 5663/474986 lines since contained in test set.


,source_sentence,target_sentence
0,Using Ladders — Do You Make These Safety Checks ?,Lílo Àkàbà — Ǹjẹ́ O Máa Ń Ṣe Àyẹ̀wò Wọ̀nyí Tó...
1,By Awake !,Látọwọ́ akọ̀ròyìn Jí !
2,correspondent in Ireland,ní Ireland
3,PAUL needed to change a bulb in an outside lig...,PAUL fẹ́ pààrọ̀ gílóòbù iná tó wà lóde ilé ẹ̀ .
4,He also needed to clean the outside upstairs w...,Ó tún fẹ́ nu àwọn fèrèsé pẹ̀tẹ́ẹ̀sì lọ́wọ́ ita...


## Pre-processing and export

It is generally a good idea to remove duplicate translations and conflicting translations from the corpus. In practice, these public corpora include some number of these that need to be cleaned.

In addition we will split our data into dev/test/train and export to the filesystem.

In [13]:
# drop duplicate translations
df_pp = df.drop_duplicates()

# drop conflicting translations
# (this is optional and something that you might want to comment out 
# depending on the size of your corpus)
df_pp.drop_duplicates(subset='source_sentence', inplace=True)
df_pp.drop_duplicates(subset='target_sentence', inplace=True)

# Shuffle the data to remove bias in dev set selection.
df_pp = df_pp.sample(frac=1, random_state=seed).reset_index(drop=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [14]:
# Install fuzzy wuzzy to remove "almost duplicate" sentences in the
# test and training sets.
! pip install fuzzywuzzy
! pip install python-Levenshtein
import time
from fuzzywuzzy import process
import numpy as np

# reset the index of the training set after previous filtering
df_pp.reset_index(drop=False, inplace=True)

# Remove samples from the training data set if they "almost overlap" with the
# samples in the test set.

# Filtering function. Adjust pad to narrow down the candidate matches to
# within a certain length of characters of the given sample.
def fuzzfilter(sample, candidates, pad):
  candidates = [x for x in candidates if len(x) <= len(sample)+pad and len(x) >= len(sample)-pad] 
  if len(candidates) > 0:
    return process.extractOne(sample, candidates)[1]
  else:
    return np.nan

# NOTE - This might run slow depending on the size of your training set. We are
# printing some information to help you track how long it would take. 
scores = []
start_time = time.time()
for idx, row in df_pp.iterrows():
  scores.append(fuzzfilter(row['source_sentence'], list(en_test_sents), 5))
  if idx % 1000 == 0:
    hours, rem = divmod(time.time() - start_time, 3600)
    minutes, seconds = divmod(rem, 60)
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds), "%0.2f percent complete" % (100.0*float(idx)/float(len(df_pp))))

# Filter out "almost overlapping samples"
df_pp['scores'] = scores
df_pp = df_pp[df_pp['scores'] < 95]

     |████████████████████████████████| 51kB 4.1MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144676 sha256=9ea934a16ebb4c02728ca1af20466e179eacf3412c08ad8bb0bde144bf075211
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein
00:00:00.13 0.00 percent complete
00:00:18.96 0.24 percent complete
00:00:38.11 0.48 percent complete
00:00:57.26 0.72 percent complete
00:01:16.08 0.96 percent complete
00:01:35.18 1.20 percent complete
00:01:53.15 1.43 percent complete
00:02:12.25 1.67 percent complete
00:02:30.91 1.91 percent complete
00:02:49.58 2.15 percent complete
00:03:09.08 2.39 percent complete
00:03:27.03 2.63 percent complete
00:03:46.28 2.87 percent complete
00:04:05.31 3.11 percent complete
00:04:24.30 3.35 percent complete
00:04:43.81 3.59 percent complete
00:05:02.76 3.82 percent complete
00:05:21.41 4.06 percent complete
00

00:07:53.90 5.98 percent complete
00:08:14.12 6.22 percent complete
00:08:32.88 6.45 percent complete
00:08:52.02 6.69 percent complete
00:09:10.59 6.93 percent complete
00:09:29.52 7.17 percent complete
00:09:49.28 7.41 percent complete
00:10:07.99 7.65 percent complete
00:10:26.80 7.89 percent complete
00:10:46.56 8.13 percent complete
00:11:05.10 8.37 percent complete
00:11:23.38 8.61 percent complete
00:11:42.67 8.84 percent complete
00:12:01.24 9.08 percent complete
00:12:20.04 9.32 percent complete
00:12:39.18 9.56 percent complete


00:12:58.06 9.80 percent complete
00:13:17.51 10.04 percent complete
00:13:36.53 10.28 percent complete
00:13:55.26 10.52 percent complete
00:14:13.70 10.76 percent complete
00:14:32.82 11.00 percent complete
00:14:52.21 11.24 percent complete
00:15:11.09 11.47 percent complete
00:15:29.42 11.71 percent complete
00:15:49.31 11.95 percent complete
00:16:08.94 12.19 percent complete
00:16:28.33 12.43 percent complete
00:16:47.36 12.67 percent complete
00:17:06.35 12.91 percent complete
00:17:24.93 13.15 percent complete
00:17:43.95 13.39 percent complete
00:18:02.92 13.63 percent complete
00:18:22.58 13.86 percent complete


00:18:42.03 14.10 percent complete
00:19:00.45 14.34 percent complete


00:19:18.89 14.58 percent complete
00:19:37.86 14.82 percent complete
00:19:56.60 15.06 percent complete
00:20:15.05 15.30 percent complete
00:20:33.81 15.54 percent complete
00:20:53.12 15.78 percent complete
00:21:13.15 16.02 percent complete
00:21:31.56 16.26 percent complete
00:21:50.40 16.49 percent complete
00:22:09.28 16.73 percent complete
00:22:28.38 16.97 percent complete
00:22:47.48 17.21 percent complete
00:23:06.20 17.45 percent complete
00:23:25.05 17.69 percent complete
00:23:44.09 17.93 percent complete
00:24:02.94 18.17 percent complete
00:24:20.85 18.41 percent complete
00:24:39.77 18.65 percent complete
00:24:58.11 18.88 percent complete
00:25:16.43 19.12 percent complete
00:25:34.91 19.36 percent complete
00:25:54.92 19.60 percent complete
00:26:13.85 19.84 percent complete
00:26:32.60 20.08 percent complete
00:26:50.91 20.32 percent complete
00:27:09.33 20.56 percent complete
00:27:28.38 20.80 percent complete


...

01:50:51.58 82.95 percent complete
01:51:10.16 83.19 percent complete
01:51:30.08 83.43 percent complete
01:51:49.02 83.67 percent complete
01:52:08.52 83.90 percent complete
01:52:27.83 84.14 percent complete


01:52:46.89 84.38 percent complete
01:53:05.78 84.62 percent complete
01:53:24.71 84.86 percent complete
01:53:43.96 85.10 percent complete
01:54:04.22 85.34 percent complete
01:54:23.32 85.58 percent complete
01:54:43.29 85.82 percent complete
01:55:02.79 86.06 percent complete
01:55:22.21 86.30 percent complete
01:55:41.33 86.53 percent complete
01:56:00.23 86.77 percent complete
01:56:19.12 87.01 percent complete
01:56:38.77 87.25 percent complete
01:56:57.61 87.49 percent complete
01:57:15.84 87.73 percent complete
01:57:34.60 87.97 percent complete
01:57:53.46 88.21 percent complete
01:58:11.84 88.45 percent complete
01:58:30.89 88.69 percent complete
01:58:49.28 88.92 percent complete


01:59:08.05 89.16 percent complete
01:59:27.61 89.40 percent complete
01:59:46.54 89.64 percent complete
02:00:05.68 89.88 percent complete
02:00:24.65 90.12 percent complete
02:00:43.91 90.36 percent complete
02:01:02.03 90.60 percent complete
02:01:20.91 90.84 percent complete
02:01:40.07 91.08 percent complete
02:01:59.19 91.32 percent complete
02:02:18.00 91.55 percent complete
02:02:36.68 91.79 percent complete
02:02:55.43 92.03 percent complete
02:03:14.24 92.27 percent complete
02:03:33.75 92.51 percent complete
02:03:52.57 92.75 percent complete
02:04:11.93 92.99 percent complete
02:04:31.85 93.23 percent complete
02:04:51.23 93.47 percent complete
02:05:10.88 93.71 percent complete
02:05:29.89 93.94 percent complete
02:05:48.95 94.18 percent complete
02:06:07.35 94.42 percent complete
02:06:26.48 94.66 percent complete
02:06:45.32 94.90 percent complete
02:07:05.28 95.14 percent complete
02:07:23.59 95.38 percent complete


02:07:42.84 95.62 percent complete
02:08:01.06 95.86 percent complete
02:08:19.53 96.10 percent complete
02:08:38.57 96.34 percent complete
02:08:57.26 96.57 percent complete
02:09:15.59 96.81 percent complete
02:09:35.51 97.05 percent complete
02:09:54.22 97.29 percent complete


02:10:13.11 97.53 percent complete
02:10:32.11 97.77 percent complete
02:10:50.97 98.01 percent complete
02:11:09.38 98.25 percent complete
02:11:28.00 98.49 percent complete
02:11:46.41 98.73 percent complete
02:12:06.18 98.96 percent complete
02:12:24.79 99.20 percent complete
02:12:43.89 99.44 percent complete
02:13:02.72 99.68 percent complete
02:13:21.36 99.92 percent complete


In [15]:
# This section does the split between train/dev for the parallel corpora then saves them as separate files
# We use 1000 dev test and the given test set.
import csv

# Do the split between dev/train and create parallel corpora
num_dev_patterns = 1000

# Optional: lower case the corpora - this will make it easier to generalize, but without proper casing.
if lc:  # Julia: making lowercasing optional
    df_pp["source_sentence"] = df_pp["source_sentence"].str.lower()
    df_pp["target_sentence"] = df_pp["target_sentence"].str.lower()

# Julia: test sets are already generated
dev = df_pp.tail(num_dev_patterns) # Herman: Error in original
stripped = df_pp.drop(df_pp.tail(num_dev_patterns).index)

with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
  for index, row in stripped.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")
    
with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  for index, row in dev.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

#stripped[["source_sentence"]].to_csv("train."+source_language, header=False, index=False)  # Herman: Added `header=False` everywhere
#stripped[["target_sentence"]].to_csv("train."+target_language, header=False, index=False)  # Julia: Problematic handling of quotation marks.

#dev[["source_sentence"]].to_csv("dev."+source_language, header=False, index=False)
#dev[["target_sentence"]].to_csv("dev."+target_language, header=False, index=False)

# Doublecheck the format below. There should be no extra quotation marks or weird characters.
! head train.*
! head dev.*

==> train.en <==
TRAIN YOUR CHILDREN : “ I teach my children to check the expiration date of any packaged food items , such as snacks , before they buy them . ” ​ — Ruth , Nigeria
When she replied that she was , he explained that he and his mother were trying to assist his sister with a school report on Canadians .
Through the prophet Zephaniah , Jehovah answers : “ That day is a day of fury , a day of distress and of anguish , a day of storm and of desolation , a day of darkness and of gloominess , a day of clouds and of thick gloom . ”
We do not require that people simply do as we tell them , but we give them convincing reasons to obey Christ’s command .
Still , Jehovah can annihilate any rebel in the lake of fire , denying him any hope of a resurrection .
Lucaris was arrested , and on July 27 , 1638 , he was taken on board a small boat as if for banishment .
Yes , Jehovah remembered their faithful course .
□ To appear tough
During the ensuing confrontation , we Witnesses had to make



---


## Installation of JoeyNMT

JoeyNMT is a simple, minimalist NMT package which is useful for learning and teaching. Check out the documentation for JoeyNMT [here](https://joeynmt.readthedocs.io)  

In [16]:
# Install JoeyNMT
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .

Cloning into 'joeynmt'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 2204 (delta 8), reused 5 (delta 3), pack-reused 2184
Receiving objects: 100% (2204/2204), 2.60 MiB | 4.31 MiB/s, done.
Resolving deltas: 100% (1529/1529), done.
Processing /content/joeynmt
     |████████████████████████████████| 276kB 24.3MB/s 
     |████████████████████████████████| 307kB 28.9MB/s 
     |████████████████████████████████| 215kB 22.3MB/s 
     |████████████████████████████████| 51kB 9.8MB/s 
     |████████████████████████████████| 747kB 38.7MB/s 
     |████████████████████████████████| 61kB 10.4MB/s 
  Created wheel for joeynmt: filename=joeynmt-0.0.1-cp36-none-any.whl size=72136 sha256=236b35fbed374cc299b613e59279e93eefdca931f670a7a4077444f9f1ef1eb9
  Stored in directory: /tmp/pip-ephem-wheel-cache-wgoxy3ml/wheels/db/01/db/751cc9f3e7f6faec127c43644ba250a3ea7ad200594aeda70a
  Created wheel for pyyam

# Preprocessing the Data into Subword BPE Tokens

- One of the most powerful improvements for agglutinative languages (a feature of most Bantu languages) is using BPE tokenization [ (Sennrich, 2015) ](https://arxiv.org/abs/1508.07909).

- It was also shown that by optimizing the number of BPE codes we significantly improve results for low-resourced languages [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021) [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

- Below we have the scripts for doing BPE tokenization of our data. We use 4000 tokens as recommended by [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021). You do not need to change anything. Simply running the below will be suitable. 

In [17]:
# One of the huge boosts in NMT performance was to use a different method of tokenizing. 
# Usually, NMT would tokenize by words. However, using a method called BPE gave amazing boosts to performance

# Do subword NMT
from os import path
os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language

# Learn BPEs on the training data.
os.environ["data_path"] = path.join("joeynmt", "data", source_language + target_language) # Herman! 
! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

# Apply BPE splits to the development and test data.
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

# Create directory, move everyone we care about to the correct location
! mkdir -p $data_path
! cp train.* $data_path
! cp test.* $data_path
! cp dev.* $data_path
! cp bpe.codes.4000 $data_path
! ls $data_path

# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

# Create that vocab using build_vocab
! sudo chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py joeynmt/data/$src$tgt/train.bpe.$src joeynmt/data/$src$tgt/train.bpe.$tgt --output_path joeynmt/data/$src$tgt/vocab.txt

# Some output
! echo "BPE Xhosa Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 joeynmt/data/$src$tgt/vocab.txt  # Herman

bpe.codes.4000	dev.en	     test.bpe.yo     test.yo	   train.en
dev.bpe.en	dev.yo	     test.en	     train.bpe.en  train.yo
dev.bpe.yo	test.bpe.en  test.en-any.en  train.bpe.yo
bpe.codes.4000	dev.en	     test.bpe.yo     test.yo	   train.en
dev.bpe.en	dev.yo	     test.en	     train.bpe.en  train.yo
dev.bpe.yo	test.bpe.en  test.en-any.en  train.bpe.yo
BPE Xhosa Sentences
A@@ p@@ at@@ a ńlá ti ìgbàgbọ́ ( Wo ìpín@@ rọ̀ 12 - 14 )
À@@ ṣí@@ borí ìgb@@ àlà ( Wo ìpín@@ rọ̀ 15 - 18 )
Mo ti rí i pé àwọn èèyàn máa ń fẹ́ gb@@ ọ́@@ rọ̀ wa tí wọ́n bá rí i pé a lóye Bíbélì dáadáa , a sì fẹ́ ran àwọn lọ́wọ́ . ”
I@@ dà ẹ̀mí ( Wo ìpín@@ rọ̀ 19 - 20 )
Ó dájú pé lọ́@@ lá ìt@@ ì@@ lẹ́yìn Jèhófà , a máa dúró gb@@ ọ@@ in - in , Èṣù ò sì ní rí wa gbé ṣe .
Combined BPE Vocab
œ@@
Ísír@@
Isra@@
̃
×
ô
ʺ
bítì
Pété@@
Jóò@@


In [18]:
# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

bpe.codes.4000	dev.en	     test.bpe.yo     test.yo	   train.en
dev.bpe.en	dev.yo	     test.en	     train.bpe.en  train.yo
dev.bpe.yo	test.bpe.en  test.en-any.en  train.bpe.yo


# Creating the JoeyNMT Config

JoeyNMT requires a yaml config. We provide a template below. We've also set a number of defaults with it, that you may play with!

- We used Transformer architecture 
- We set our dropout to reasonably high: 0.3 (recommended in  [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021))

Things worth playing with:
- The batch size (also recommended to change for low-resourced languages)
- The number of epochs (we've set it at 30 just so it runs in about an hour, for testing purposes)
- The decoder options (beam_size, alpha)
- Evaluation metrics (BLEU versus Crhf4)

In [0]:
# This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# (You can of course play with all the parameters if you'd like!)

name = '%s%s' % (source_language, target_language)
gdrive_path = os.environ["gdrive_path"]

# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "data/{name}/train.bpe"
    dev:   "data/{name}/dev.bpe"
    test:  "data/{name}/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "data/{name}/vocab.txt"
    trg_vocab: "data/{name}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    #load_model: "{gdrive_path}/models/{name}_transformer/1.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 30                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 1000          # TODO: Set to at least once per epoch.
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "models/{name}_transformer"
    overwrite: False               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(name=name, gdrive_path=os.environ["gdrive_path"], source_language=source_language, target_language=target_language)
with open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

# Train the Model

This single line of joeynmt runs the training using the config we made above

In [0]:
# Train the model
# You can press Ctrl-C to stop. And then run the next cell to save your checkpoints! 
!cd joeynmt; python3 -m joeynmt train configs/transformer_$src$tgt.yaml

2020-01-17 20:02:04,366 Hello! This is Joey-NMT.
2020-01-17 20:02:05,435 Total params: 12188160
2020-01-17 20:02:05,436 Trainable parameters: ['decoder.layer_norm.bias', 'decoder.layer_norm.weight', 'decoder.layers.0.dec_layer_norm.bias', 'decoder.layers.0.dec_layer_norm.weight', 'decoder.layers.0.feed_forward.layer_norm.bias', 'decoder.layers.0.feed_forward.layer_norm.weight', 'decoder.layers.0.feed_forward.pwff_layer.0.bias', 'decoder.layers.0.feed_forward.pwff_layer.0.weight', 'decoder.layers.0.feed_forward.pwff_layer.3.bias', 'decoder.layers.0.feed_forward.pwff_layer.3.weight', 'decoder.layers.0.src_trg_att.k_layer.bias', 'decoder.layers.0.src_trg_att.k_layer.weight', 'decoder.layers.0.src_trg_att.output_layer.bias', 'decoder.layers.0.src_trg_att.output_layer.weight', 'decoder.layers.0.src_trg_att.q_layer.bias', 'decoder.layers.0.src_trg_att.q_layer.weight', 'decoder.layers.0.src_trg_att.v_layer.bias', 'decoder.layers.0.src_trg_att.v_layer.weight', 'decoder.layers.0.trg_trg_att.k_l

In [0]:
# Copy the created models from the notebook storage to google drive for persistant storage 
!cp -r joeynmt/models/${src}${tgt}_transformer/* "$gdrive_path/models/${src}${tgt}_transformer/"

In [0]:
# Output our validation accuracy
! cat "$gdrive_path/models/${src}${tgt}_transformer/validations.txt"

In [0]:
# Test our model
! cd joeynmt; python3 -m joeynmt test "$gdrive_path/models/${src}${tgt}_transformer/config.yaml"